In [ ]:
import numpy as np
import pandas as pd
import random

In [ ]:
win_prob = pd.read_csv('game_win_probabilities.csv')
win_prob.head()

,Away Team,Home Team,Home Team Win Probability
0,Baltimore Ravens,Kansas City Chiefs,0.500000
1,Green Bay Packers,Philadelphia Eagles,0.500000
2,Pittsburgh Steelers,Atlanta Falcons,0.527778
3,Arizona Cardinals,Buffalo Bills,0.611111
4,Tennessee Titans,Chicago Bears,0.680000


In [ ]:
# Import other tables
records = pd.read_csv('team_records.csv', index_col='team')
divisions = pd.read_csv('nfl_divisions.csv')
divisions.set_index('Team', inplace=True)

In [ ]:
division_names = ['NFC North', 'NFC South', 'NFC East', 'NFC West', 'AFC North', 'AFC South', 'AFC East', 'AFC West']

In [ ]:
results = pd.DataFrame(columns =  division_names + ['NFC Wild Cards', 'AFC Wild Cards'])

## A Single Simulation

In [ ]:
random.seed(31659)

for season in range(10000):
    if season % 500 == 0:
        print(f"Running Season {season}")

    records = pd.read_csv('team_records.csv', index_col='team')
    win_prob['Winner'] = ''

    for i in range(win_prob.shape[0]):
        home_team = win_prob.loc[i, 'Home Team']
        away_team = win_prob.loc[i, 'Away Team']
        if random.random() < win_prob.loc[i, 'Home Team Win Probability']:
            win_prob.loc[i, 'Winner'] = home_team
            records.loc[away_team, 'Losses'] += 1
            records.loc[home_team, 'Wins'] += 1
            if divisions.loc[away_team, 'Conference'] == divisions.loc[home_team, 'Conference']:
                records.loc[away_team, 'Conf Losses'] += 1
                records.loc[home_team, 'Conf Wins'] += 1
                if divisions.loc[away_team, 'Division'] == divisions.loc[home_team, 'Division']:
                    records.loc[away_team, 'Div Losses'] += 1
                    records.loc[home_team, 'Div Wins'] += 1

        else:
            win_prob.loc[i, 'Winner'] = away_team
            records.loc[away_team, 'Wins'] += 1
            records.loc[home_team, 'Losses'] += 1
            if divisions.loc[away_team, 'Conference'] == divisions.loc[home_team, 'Conference']:
                records.loc[away_team, 'Conf Wins'] += 1
                records.loc[home_team, 'Conf Losses'] += 1
                if divisions.loc[away_team, 'Division'] == divisions.loc[home_team, 'Division']:
                    records.loc[away_team, 'Div Wins'] += 1
                    records.loc[home_team, 'Div Losses'] += 1

    combined = records.join(divisions)

    # Add an empty row to add season results
    results = pd.concat([results, pd.DataFrame([['']*results.shape[1]], columns = results.columns)], ignore_index=True)

    def process_division_winner(division, winner):
        results.loc[season, division] = winner
        combined.drop(index=winner, inplace=True)


    for division in division_names:
        div_results = combined.loc[combined['Combined'] == division, :].sort_values(by='Wins', ascending=False)

        # Check for tie in division standings
        if div_results.iloc[0, 0] == div_results.iloc[1, 0]:
            # Check Head to Head record
            h2h_1_winner = win_prob.loc[(win_prob['Away Team'] == div_results.index[0]) & (win_prob['Home Team'] == div_results.index[1]), 'Winner'].squeeze()
            h2h_2_winner = win_prob.loc[(win_prob['Away Team'] == div_results.index[1]) & (win_prob['Home Team'] == div_results.index[0]), 'Winner'].squeeze()
            if h2h_1_winner != h2h_2_winner:
                # If tied, get better division record
                if div_results.iloc[0, 4] > div_results.iloc[1, 4]:
                    process_division_winner(division, div_results.index[0])
                elif div_results.iloc[0, 4] < div_results.iloc[1, 4]:
                    process_division_winner(division, div_results.index[1])
                else:
                    # If tied, get better conference record (Note: I skipped the common games tiebreaker for simplicity)
                    if div_results.iloc[0, 2] > div_results.iloc[1, 2]:
                        process_division_winner(division, div_results.index[0])
                    elif div_results.iloc[0, 2] < div_results.iloc[1, 2]:
                        process_division_winner(division, div_results.index[1])
                    else:
                        # results.loc[season, division] = f"{div_results.index[0]} and {div_results.index[1]} ({div_results.iloc[0, 0]}-{div_results.iloc[0, 1]})"
                        results.loc[season, division] = f"{div_results.index[0]} and {div_results.index[1]}"
                        combined.drop(index=div_results.index[0], inplace=True)
                        combined.drop(index=div_results.index[1], inplace=True)

            else:
                process_division_winner(division, h2h_1_winner)

        else:
            process_division_winner(division, div_results.index[0])


    nfc_standings = combined[combined['Conference'] == 'NFC'].sort_values(by='Wins', ascending=False)
    nfc_wild = list(nfc_standings.index[:3])
    if nfc_standings.iloc[2, 0] == nfc_standings.iloc[3,0]:
        wins = nfc_standings.iloc[2, 0]
        idx = 3
        while nfc_standings.iloc[idx, 0] == wins:
            nfc_wild.append(nfc_standings.index[idx])
            idx += 1

    afc_standings = combined[combined['Conference'] == 'AFC'].sort_values(by='Wins', ascending=False)
    afc_wild = list(afc_standings.index[:3])
    if afc_standings.iloc[2, 0] == afc_standings.iloc[3,0]:
        wins = afc_standings.iloc[2, 0]
        idx = 3
        while afc_standings.iloc[idx, 0] == wins:
            afc_wild.append(afc_standings.index[idx])
            idx += 1

    results.loc[season, 'NFC Wild Cards'] = nfc_wild
    results.loc[season, 'AFC Wild Cards'] = afc_wild

Running Season 0
Running Season 500
Running Season 1000
Running Season 1500
Running Season 2000
Running Season 2500
Running Season 3000
Running Season 3500
Running Season 4000
Running Season 4500
Running Season 5000
Running Season 5500
Running Season 6000
Running Season 6500
Running Season 7000
Running Season 7500
Running Season 8000
Running Season 8500
Running Season 9000
Running Season 9500


In [ ]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

In [ ]:
results.head()

,NFC North,NFC South,NFC East,NFC West,AFC North,AFC South,AFC East,AFC West,NFC Wild Cards,AFC Wild Cards
0,Green Bay Packers,New Orleans Saints,Dallas Cowboys,San Francisco 49ers,Baltimore Ravens,Jacksonville Jaguars,New York Jets,Kansas City Chiefs,"[Los Angeles Rams, Atlanta Falcons, Arizona Cardinals, Tampa Bay Buccaneers]","[Houston Texans, Buffalo Bills, Cleveland Browns]"
1,Chicago Bears,Atlanta Falcons,Washington Commanders,San Francisco 49ers,Cincinnati Bengals and Pittsburgh Steelers,Indianapolis Colts,New York Jets,Kansas City Chiefs,"[Detroit Lions, Philadelphia Eagles, Los Angeles Rams]","[Houston Texans, Buffalo Bills, Los Angeles Chargers]"
2,Minnesota Vikings,Atlanta Falcons,Philadelphia Eagles,San Francisco 49ers,Pittsburgh Steelers,Indianapolis Colts,Miami Dolphins,Kansas City Chiefs,"[Green Bay Packers, Seattle Seahawks, Chicago Bears, New Orleans Saints]","[Baltimore Ravens, Buffalo Bills, Cincinnati Bengals, Cleveland Browns]"
3,Detroit Lions,Tampa Bay Buccaneers,Dallas Cowboys,San Francisco 49ers,Cincinnati Bengals,Jacksonville Jaguars,Buffalo Bills,Las Vegas Raiders,"[Chicago Bears, Philadelphia Eagles, Green Bay Packers, Minnesota Vikings, New York Giants]","[Miami Dolphins, Baltimore Ravens, Kansas City Chiefs, Los Angeles Chargers]"
4,Green Bay Packers,Tampa Bay Buccaneers,Philadelphia Eagles,Los Angeles Rams,Baltimore Ravens,Houston Texans,Buffalo Bills,Los Angeles Chargers,"[Chicago Bears, Dallas Cowboys, Seattle Seahawks]","[Kansas City Chiefs, Cincinnati Bengals, Cleveland Browns, Indianapolis Colts, Jacksonville Jaguars, Miami Dolphins, Pittsburgh Steelers]"


In [ ]:
for division in division_names:
    print(f"{division} Results:\n{results[division].value_counts()}\n\n")

NFC North Results:
Detroit Lions                              4203
Green Bay Packers                          3003
Chicago Bears                              1954
Minnesota Vikings                           709
Detroit Lions and Green Bay Packers          42
Chicago Bears and Detroit Lions              38
Chicago Bears and Green Bay Packers          25
Detroit Lions and Minnesota Vikings          11
Green Bay Packers and Minnesota Vikings       9
Chicago Bears and Minnesota Vikings           6
Name: NFC North, dtype: int64


NFC South Results:
Atlanta Falcons                                4757
Tampa Bay Buccaneers                           2883
New Orleans Saints                             1990
Carolina Panthers                               248
Atlanta Falcons and New Orleans Saints           47
Atlanta Falcons and Tampa Bay Buccaneers         46
New Orleans Saints and Tampa Bay Buccaneers      19
Atlanta Falcons and Carolina Panthers             4
Carolina Panthers and New Orleans 

Totals of different combinations of division winners (top 75 combinations).

In [ ]:
combos = results.groupby(['AFC West', 'NFC West', 'NFC South', 'AFC South', 'NFC East', 'NFC North', 'AFC East', 'AFC North']).count().sort_values('NFC Wild Cards', ascending=False)
combos.drop(columns = 'AFC Wild Cards', inplace=True)
combos.columns = ['Count']
combos[:50]

Count
AFC West           NFC West            NFC South            AFC South            NFC East            NFC North         AFC East       AFC North                 
Kansas City Chiefs San Francisco 49ers Atlanta Falcons      Houston Texans       Philadelphia Eagles Detroit Lions     New York Jets  Cincinnati Bengals      30
                                                                                                                                      Baltimore Ravens        28
                                                                                                                       Buffalo Bills  Baltimore Ravens        26
                                                                                                                                      Cincinnati Bengals      23
                                                                                 Dallas Cowboys      Green Bay Packers New York Jets  Cincinnati Bengals      20
                                                                                 Philadelphia Eagles Green Bay Packers Buffalo Bills  Cincinnati Bengals      20
                                                                                                                       New York Jets  Cincinnati Bengals      20
                                       New Orleans Saints   Houston Texans       Philadelphia Eagles Detroit Lions     Buffalo Bills  Cincinnati Bengals      18
                                       Atlanta Falcons      Indianapolis Colts   Philadelphia Eagles Detroit Lions     New York Jets  Baltimore Ravens        17
                                                            Houston Texans       Dallas Cowboys      Green Bay Packers Buffalo Bills  Cincinnati Bengals      16
                                                                                 Philadelphia Eagles Detroit Lions     Buffalo Bills  Cleveland Browns        16
                                                                                 Dallas Cowboys      Detroit Lions     Buffalo Bills  Baltimore Ravens        16
                                       Tampa Bay Buccaneers Houston Texans       Philadelphia Eagles Detroit Lions     New York Jets  Baltimore Ravens        16
                                       Atlanta Falcons      Houston Texans       Dallas Cowboys      Detroit Lions     New York Jets  Cincinnati Bengals      15
                                                                                 Philadelphia Eagles Detroit Lions     Miami Dolphins Baltimore Ravens        15
                   Los Angeles Rams    Atlanta Falcons      Houston Texans       Dallas Cowboys      Detroit Lions     New York Jets  Baltimore Ravens        15
                   San Francisco 49ers Atlanta Falcons      Jacksonville Jaguars Philadelphia Eagles Detroit Lions     Buffalo Bills  Cincinnati Bengals      15
                                       Tampa Bay Buccaneers Houston Texans       Philadelphia Eagles Detroit Lions     Buffalo Bills  Cincinnati Bengals      14
                                       Atlanta Falcons      Houston Texans       Philadelphia Eagles Green Bay Packers Miami Dolphins Cincinnati Bengals      14
                   Los Angeles Rams    Atlanta Falcons      Houston Texans       Philadelphia Eagles Detroit Lions     Buffalo Bills  Cincinnati Bengals      14
                                                                                                                       Miami Dolphins Cincinnati Bengals      13
                   San Francisco 49ers Atlanta Falcons      Houston Texans       Dallas Cowboys      Chicago Bears     New York Jets  Baltimore Ravens        13
                                       Tampa Bay Buccaneers Houston Texans       Philadelphia Eagles Detroit Lions     Buffalo Bills  Baltimore Ravens        13
                                       Atlanta Falcons      Jacksonville Jaguars Philadelphia Eagles Detroit Lions     Miami Dolph

# Summarizing the results

We can see that the model is very confident about most of the division winners:
* AFC West: Kansas City Chiefs
* NFC West: San Francisco 49ers
* NFC South: Atlanta Falcons
* AFC South: Houston Texans
* NFC East: Philadelphia Eagles
* NFC North: Detroit Lions

However, it seems like two of the divisions may be a toss up, with the Buffalo Bills and New York Jets fighting for the AFC East title and the Baltimore Ravens and Cincinnati Bengals duking it out in the AFC North. To determine the most likely result, we will subset the simulations based on what we think will happen in the other 6 divisions and see if there is a more stark contrast in the results.

In [ ]:
combos.index.nlevels

8

In [ ]:
combos.loc['Kansas City Chiefs', 'San Francisco 49ers', 'Atlanta Falcons', 'Houston Texans', 'Philadelphia Eagles', 'Detroit Lions', :, :, combos['Count'] >= 5]

Count
AFC West           NFC West            NFC South       AFC South      NFC East            NFC North     AFC East       AFC North                 
Kansas City Chiefs San Francisco 49ers Atlanta Falcons Houston Texans Philadelphia Eagles Detroit Lions New York Jets  Cincinnati Bengals      30
                                                                                                                       Baltimore Ravens        28
                                                                                                        Buffalo Bills  Baltimore Ravens        26
                                                                                                                       Cincinnati Bengals      23
                                                                                                                       Cleveland Browns        16
                                                                                                        Miami Dolphins Baltimore Ravens        15
                                                                                                        New York Jets  Cleveland Browns        13
                                                                                                        Buffalo Bills  Pittsburgh Steelers     10
                                                                                                        Miami Dolphins Cincinnati Bengals       8
                                                                                                                       Cleveland Browns         8
                                                                                                        New York Jets  Pittsburgh Steelers      5

In [ ]:
combos.loc['Kansas City Chiefs', 'San Francisco 49ers', 'Atlanta Falcons', 'Houston Texans', 'Philadelphia Eagles', 'Detroit Lions', 'New York Jets', :, :].sum()

Count    77
dtype: int64

In [ ]:
combos.loc['Kansas City Chiefs', 'San Francisco 49ers', 'Atlanta Falcons', 'Houston Texans', 'Philadelphia Eagles', 'Detroit Lions', 'Buffalo Bills', :, :].sum()

Count    76
dtype: int64

In [ ]:
combos.loc['Kansas City Chiefs', 'San Francisco 49ers', 'Atlanta Falcons', 'Houston Texans', 'Philadelphia Eagles', 'Detroit Lions', 'Miami Dolphins', :, :].sum()

Count    34
dtype: int64

In [ ]:
combos.loc['Kansas City Chiefs', 'San Francisco 49ers', 'Atlanta Falcons', 'Houston Texans', 'Philadelphia Eagles', 'Detroit Lions', :, 'Baltimore Ravens', :].sum()

Count    73
dtype: int64

In [ ]:
combos.loc['Kansas City Chiefs', 'San Francisco 49ers', 'Atlanta Falcons', 'Houston Texans', 'Philadelphia Eagles', 'Detroit Lions', :, 'Cincinnati Bengals', :].sum()

Count    65
dtype: int64

In [ ]:
combos.loc['Kansas City Chiefs', 'San Francisco 49ers', 'Atlanta Falcons', 'Houston Texans', 'Philadelphia Eagles', 'Detroit Lions', :, 'Cleveland Browns', :].sum()

Count    42
dtype: int64

Assuming the results in the other 6 divisions, the AFC East and AFC North seem like they may be toss ups this year. The model seems to give a slight edge to the New York Jets in the AFC East and the Ravens in the AFC North, but the Buffalo Bills and Cincinnati Bengals are right there as well. Another dark horse that emerged from the AFC North is the Cleveland Browns, who won the division in about 23% of these simulations.

Now let's see who the most likely wild card teams are given these division winners.

In [ ]:
predictions = results.loc[
    (results['NFC North'] == 'Detroit Lions') &
    (results['NFC South'] == 'Atlanta Falcons') & 
    (results['NFC East'] == 'Philadelphia Eagles') &
    (results['NFC West'] == 'San Francisco 49ers') &
    (results['AFC North'] == 'Baltimore Ravens') &
    (results['AFC South'] == 'Houston Texans') &
    (results['AFC East'] == 'New York Jets') &
    (results['AFC West'] == 'Kansas City Chiefs'), :
].reset_index(drop=True)

In [ ]:
nfc_wild = {
    'Arizona Cardinals': 0,
    'Carolina Panthers': 0,
    'Chicago Bears': 0,
    'Dallas Cowboys': 0,
    'Green Bay Packers': 0,
    'Los Angeles Rams': 0,
    'Minnesota Vikings': 0,
    'New Orleans Saints': 0,
    'New York Giants': 0,
    'Seattle Seahawks': 0,
    'Tampa Bay Buccaneers': 0,
    'Washington Commanders': 0
}

for sim in predictions['NFC Wild Cards']:
    for team in sim:
        nfc_wild[team] += 1

In [ ]:
afc_wild = {
    'Buffalo Bills': 0,
    'Cincinnati Bengals': 0,
    'Cleveland Browns': 0,
    'Denver Broncos': 0,
    'Indianapolis Colts': 0,
    'Jacksonville Jaguars': 0,
    'Las Vegas Raiders': 0,
    'Los Angeles Chargers': 0,
    'Miami Dolphins': 0,
    'New England Patriots': 0,
    'Pittsburgh Steelers': 0,
    'Tennessee Titans': 0
}

for sim in predictions['AFC Wild Cards']:
    for team in sim:
        afc_wild[team] += 1

In [ ]:
nfc_wild

{'Arizona Cardinals': 10,
 'Carolina Panthers': 2,
 'Chicago Bears': 12,
 'Dallas Cowboys': 11,
 'Green Bay Packers': 16,
 'Los Angeles Rams': 15,
 'Minnesota Vikings': 6,
 'New Orleans Saints': 9,
 'New York Giants': 0,
 'Seattle Seahawks': 13,
 'Tampa Bay Buccaneers': 5,
 'Washington Commanders': 2}

In [ ]:
afc_wild

{'Buffalo Bills': 8,
 'Cincinnati Bengals': 19,
 'Cleveland Browns': 15,
 'Denver Broncos': 3,
 'Indianapolis Colts': 10,
 'Jacksonville Jaguars': 9,
 'Las Vegas Raiders': 10,
 'Los Angeles Chargers': 3,
 'Miami Dolphins': 8,
 'New England Patriots': 2,
 'Pittsburgh Steelers': 12,
 'Tennessee Titans': 2}

In the NFC, there seem to be two leaders for wild card spot, the Green Bay Packers and Los Angeles Rams.
In the AFC, the two that stound out are the Cincinnati Bengals and Cleveland Browns. Interestingly, the Buffalo Bills earned a wild card spot less often than the several other teams, even though they won the division in many simulations.

In [ ]:
further_subset = predictions[(predictions['NFC Wild Cards'].apply(lambda x: 'Green Bay Packers' in x)) & (predictions['AFC Wild Cards'].apply(lambda x: 'Cincinnati Bengals' in x))]

In [ ]:
nfc_wild = {
    'Arizona Cardinals': 0,
    'Carolina Panthers': 0,
    'Chicago Bears': 0,
    'Dallas Cowboys': 0,
    'Green Bay Packers': 0,
    'Los Angeles Rams': 0,
    'Minnesota Vikings': 0,
    'New Orleans Saints': 0,
    'New York Giants': 0,
    'Seattle Seahawks': 0,
    'Tampa Bay Buccaneers': 0,
    'Washington Commanders': 0
}

for sim in further_subset['NFC Wild Cards']:
    for team in sim:
        nfc_wild[team] += 1

In [ ]:
afc_wild = {
    'Buffalo Bills': 0,
    'Cincinnati Bengals': 0,
    'Cleveland Browns': 0,
    'Denver Broncos': 0,
    'Indianapolis Colts': 0,
    'Jacksonville Jaguars': 0,
    'Las Vegas Raiders': 0,
    'Los Angeles Chargers': 0,
    'Miami Dolphins': 0,
    'New England Patriots': 0,
    'Pittsburgh Steelers': 0,
    'Tennessee Titans': 0
}

for sim in further_subset['AFC Wild Cards']:
    for team in sim:
        afc_wild[team] += 1

In [ ]:
nfc_wild

{'Arizona Cardinals': 2,
 'Carolina Panthers': 1,
 'Chicago Bears': 4,
 'Dallas Cowboys': 4,
 'Green Bay Packers': 11,
 'Los Angeles Rams': 5,
 'Minnesota Vikings': 2,
 'New Orleans Saints': 2,
 'New York Giants': 0,
 'Seattle Seahawks': 3,
 'Tampa Bay Buccaneers': 2,
 'Washington Commanders': 0}

In [ ]:
afc_wild

{'Buffalo Bills': 3,
 'Cincinnati Bengals': 11,
 'Cleveland Browns': 6,
 'Denver Broncos': 0,
 'Indianapolis Colts': 5,
 'Jacksonville Jaguars': 1,
 'Las Vegas Raiders': 4,
 'Los Angeles Chargers': 0,
 'Miami Dolphins': 3,
 'New England Patriots': 1,
 'Pittsburgh Steelers': 5,
 'Tennessee Titans': 1}

In [ ]:
further_subset

,NFC North,NFC South,NFC East,NFC West,AFC North,AFC South,AFC East,AFC West,NFC Wild Cards,AFC Wild Cards
0,Detroit Lions,Atlanta Falcons,Philadelphia Eagles,San Francisco 49ers,Baltimore Ravens,Houston Texans,New York Jets,Kansas City Chiefs,"[Green Bay Packers, Los Angeles Rams, Tampa Bay Buccaneers]","[Cincinnati Bengals, Indianapolis Colts, Las Vegas Raiders]"
2,Detroit Lions,Atlanta Falcons,Philadelphia Eagles,San Francisco 49ers,Baltimore Ravens,Houston Texans,New York Jets,Kansas City Chiefs,"[Dallas Cowboys, Green Bay Packers, Minnesota Vikings]","[Cincinnati Bengals, Indianapolis Colts, Pittsburgh Steelers]"
6,Detroit Lions,Atlanta Falcons,Philadelphia Eagles,San Francisco 49ers,Baltimore Ravens,Houston Texans,New York Jets,Kansas City Chiefs,"[Arizona Cardinals, Carolina Panthers, Chicago Bears, Green Bay Packers, Minnesota Vikings]","[Cleveland Browns, Pittsburgh Steelers, Cincinnati Bengals]"
7,Detroit Lions,Atlanta Falcons,Philadelphia Eagles,San Francisco 49ers,Baltimore Ravens,Houston Texans,New York Jets,Kansas City Chiefs,"[Green Bay Packers, Dallas Cowboys, Seattle Seahawks]","[Cincinnati Bengals, Jacksonville Jaguars, Las Vegas Raiders, Miami Dolphins]"
9,Detroit Lions,Atlanta Falcons,Philadelphia Eagles,San Francisco 49ers,Baltimore Ravens,Houston Texans,New York Jets,Kansas City Chiefs,"[Green Bay Packers, New Orleans Saints, Seattle Seahawks]","[Buffalo Bills, Cleveland Browns, Cincinnati Bengals, Indianapolis Colts, Miami Dolphins, Pittsburgh Steelers]"
11,Detroit Lions,Atlanta Falcons,Philadelphia Eagles,San Francisco 49ers,Baltimore Ravens,Houston Texans,New York Jets,Kansas City Chiefs,"[Green Bay Packers, Chicago Bears, Los Angeles Rams]","[Cleveland Browns, Cincinnati Bengals, Miami Dolphins]"
14,Detroit Lions,Atlanta Falcons,Philadelphia Eagles,San Francisco 49ers,Baltimore Ravens,Houston Texans,New York Jets,Kansas City Chiefs,"[Green Bay Packers, New Orleans Saints, Chicago Bears]","[Las Vegas Raiders, Pittsburgh Steelers, Buffalo Bills, Cincinnati Bengals]"
19,Detroit Lions,Atlanta Falcons,Philadelphia Eagles,San Francisco 49ers,Baltimore Ravens,Houston Texans,New York Jets,Kansas City Chiefs,"[Arizona Cardinals, Los Angeles Rams, Green Bay Packers]","[Cleveland Browns, Pittsburgh Steelers, Cincinnati Bengals]"
20,Detroit Lions,Atlanta Falcons,Philadelphia Eagles,San Francisco 49ers,Baltimore Ravens,Houston Texans,New York Jets,Kansas City Chiefs,"[Green Bay Packers, Chicago Bears, Los Angeles Rams]","[Cincinnati Bengals, Indianapolis Colts, Cleveland Browns, Tennessee Titans]"
25,Detroit Lions,Atlanta Falcons,Philadelphia Eagles,San Francisco 49ers,Baltimore Ravens,Houston Texans,New York Jets,Kansas City Chiefs,"[Green Bay Packers, Tampa Bay Buccaneers, Dallas Cowboys, Los Angeles Rams]","[Buffalo Bills, Cincinnati Bengals, Indianapolis Colts]"


With this further subsetting, the final two teams that emerge from the NFC are the Los Angeles Rams and the Chicago Bears (with the Dallas Cowboys right behind them). The AFC sees a flurry of potential wild card teams, but the last two teams that are most likely to secure a spot are the the Cleveland Browns and the Indianapolis Colts.

## Final Results

### Most Likely Result
Here is what our simulation found to be the most likely playoff picture for the 2024-2025 NFL Season:

NFC North Division Winner: Detroit Lions

NFC South Division Winner: Atlanta Falcons

NFC East Division Winner: Philadelphia Eagles

NFC West Division Winner: San Francisco 49ers

NFC Wild Cards: Green Bay Packers, Los Angeles Rams, Chicago Bears

AFC North Division Winner: Baltimore Ravens

AFC South Division Winner: Houston Texans

AFC East Division Winner: New York Jets

AFC West Division Winner: Kansas City Chiefs

AFC Wild Cards: Cincinnati Bengals, Cleveland Browns, Indianapolis Colts


### Some other interesting takeaways
The model predicts another big year for the AFC North, with 3 teams from that division making the playoffs.

The only team to get 0 wild card appearances assuming the division winners listed above was the New York Giants, so don't expect much from them this year. However, the model has not given up hope on a wild card birth for the Arizona Cardinals, who finished with just 4 wins last season (perhaps a dark horse team).

The model predicts a boom or bust year for a few teams. For example, the model views the Buffalo Bills as a contender to win the AFC East but could also see them out of the playoffs entirely. The same goes for the Dallas Cowboys in the NFC East.